In [44]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix

def generate_data(snr, n_elements):
  """Generates data for the UAV-IRS SWIPT system.

  Args:
    snr: The SNR in dB.
    n_elements: The number of elements in the IRS.

  Returns:
    The channel matrix, the received signal at the UAV, and the received signal at the receiver.
  """

  h_a = tf.random.normal(shape=(n_elements, 1))
  h_r = tf.random.normal(shape=(n_elements, 1))
  g_t = tf.random.normal(shape=(1, 1))
  noise = tf.random.normal(shape=(1, 1))

  channel_matrix = tf.zeros((1, n_elements))
  for i in range(n_elements):
    channel_matrix[0, i] = tf.math.real(h_a[i] * tf.exp(1j * np.angle(h_r[i])))

  received_signal_at_uav = channel_matrix * g_t + noise
  received_signal_at_receiver = tf.matmul(channel_matrix.T, tf.exp(1j * np.pi * np.arange(n_elements) / n_elements)) * g_t + noise

  return channel_matrix, received_signal_at_uav, received_signal_at_receiver




def train_model(channel_matrix, received_signal_at_uav, received_signal_at_receiver):
  """Trains a machine learning model to optimize the grouping of RIS elements.

  Args:
    channel_matrix: The channel matrix.
    received_signal_at_uav: The received signal at the UAV.
    received_signal_at_receiver: The received signal at the receiver.

  Returns:
    The trained machine learning model.
  """

  model = Sequential()
  model.add(Dense(100, activation='relu'))
  model.add(Dense(100, activation='relu'))
  model.add(Dense(2, activation='softmax'))

  # Change the loss function to `sparse_categorical_crossentropy`.
  model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

  # Change the `y` argument to the labels of the RIS elements.


  


In [51]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix

def generate_data(snr, n_elements):
    """Generates data for the UAV-IRS SWIPT system.
    
    Args:
        snr: The SNR in dB.
        n_elements: The number of elements in the IRS.
    
    Returns:
        The channel matrix, the received signal at the UAV, and the received signal at the receiver.
    """

    h_a = tf.random.normal(shape=(n_elements, 1))
    h_r = tf.random.normal(shape=(n_elements, 1))
    g_t = tf.random.normal(shape=(1, 1))
    noise = tf.random.normal(shape=(1, 1))

    channel_matrix = tf.zeros((1, n_elements), dtype=tf.complex64)
    for i in range(n_elements):
        channel_matrix = tf.tensor_scatter_nd_add(channel_matrix, [[0, i]], [h_a[i] * tf.exp(1j * np.angle(h_r[i]))])

    received_signal_at_uav = tf.math.real(channel_matrix * g_t) + noise
    received_signal_at_receiver = tf.matmul(
        channel_matrix, tf.exp(1j * np.pi * np.arange(n_elements) / n_elements)
    ) * g_t + noise

    return channel_matrix, received_signal_at_uav, received_signal_at_receiver

def train_model(channel_matrix, received_signal_at_uav, received_signal_at_receiver):
    """Trains a machine learning model to optimize the grouping of RIS elements.
    
    Args:
        channel_matrix: The channel matrix.
        received_signal_at_uav: The received signal at the UAV.
        received_signal_at_receiver: The received signal at the receiver.
    
    Returns:
        The trained machine learning model.
    """

    model = Sequential()
    model.add(Dense(100, activation='relu'))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(2, activation='softmax'))

    # Change the loss function to `sparse_categorical_crossentropy`.
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    # Change the `y` argument to the labels of the RIS elements.

    return model

# Generate data
snr = 20
n_elements = 10
channel_matrix, received_signal_at_uav, received_signal_at_receiver = generate_data(snr, n_elements)

# Train the model
model = train_model(channel_matrix, received_signal_at_uav, received_signal_at_receiver)

# Visualize the training history
history = model.fit(channel_matrix, received_signal_at_uav, epochs=50, verbose=1, validation_split=0.2)
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.tight_layout()
plt.show()

# Evaluate the model
predicted_labels = model.predict_classes(channel_matrix)
true_labels = np.argmax(received_signal_at_uav, axis=1)
conf_matrix = confusion_matrix(true_labels, predicted_labels)
print("Confusion Matrix:\n", conf_matrix)
print("\nClassification Report:\n", classification_report(true_labels, predicted_labels))

# Visualize the confusion matrix
plt.figure(figsize=(6, 6))
plt.imshow(conf_matrix, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.colorbar()
plt.xticks([0, 1], ['Group 0', 'Group 1'])
plt.yticks([0, 1], ['Group 0', 'Group 1'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()


InvalidArgumentError: cannot compute Mul as input #1(zero-based) was expected to be a float tensor but is a complex64 tensor [Op:Mul] name: 